In [1]:
from datasets import load_dataset

# Import scikit-learn, seaborn
import seaborn as sns
import sklearn
import numpy as np
import pandas as pd
from sklearn.utils import resample

# Import Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
MLMA_hate_speech = load_dataset("nedjmaou/MLMA_hate_speech")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Convert to a DataFrame
MLMA_hate_speech_df = MLMA_hate_speech["train"].to_pandas()

In [4]:
# Find the NaN values in each column
for column in MLMA_hate_speech_df.columns:
    print(column, MLMA_hate_speech_df[column].isna().sum())

HITId 0
tweet 0
sentiment 0
directness 0
annotator_sentiment 0
target 0
group 0


In [5]:
# Find the value counts for each column
for column in MLMA_hate_speech_df.columns:
    print(column, MLMA_hate_speech_df[column].value_counts())

HITId HITId
3353    4
3352    4
3351    4
3350    4
3349    4
       ..
4542    2
4543    2
4531    2
5647    1
0       1
Name: count, Length: 5648, dtype: int64
tweet tweet
@user retard                                                                                               6
@user nigger                                                                                               6
@user twat                                                                                                 5
@user faggot                                                                                               4
@user @user retard                                                                                         4
                                                                                                          ..
sami wins. straight retarded shit                                                                          1
remembering retarded fifteen sixteen                           

In [6]:
# Get all unique values of annotator_sentiment
annotator_sentiment = MLMA_hate_speech_df["annotator_sentiment"].unique()

In [7]:
# Create a list to capture unique_annotator_sentiment
unique_annotator_sentiment = []
word_count = 0

# Use a for loop to capture each unique word in annotator_sentiment by parsing using "_"
for word in annotator_sentiment:
    words = word.split("_")
    for word in words:
        if word not in unique_annotator_sentiment:
            unique_annotator_sentiment.append(word)
            word_count += 1

unique_annotator_sentiment, word_count

(['shock', 'anger', 'confusion', 'sadness', 'indifference', 'disgust', 'fear'],
 7)

In [8]:
# Add features for each unique annotator sentiment word
for word in unique_annotator_sentiment:
    MLMA_hate_speech_df[f"{word}_annotator_sentiment"] = 0

In [9]:
MLMA_hate_speech_df.head()

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...,hateful_normal,indirect,shock,gender,individual,0,0,0,0,0,0,0
1,2,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other,0,0,0,0,0,0,0
2,3,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...,offensive,indirect,indifference,other,individual,0,0,0,0,0,0,0
3,4,@user @user انا اوافقك بخصوص السوريين و العراق...,normal,direct,indifference,origin,other,0,0,0,0,0,0,0
4,5,هذه السعودية التي شعبها شعب الخيم و بول البعير...,normal,indirect,indifference,origin,other,0,0,0,0,0,0,0


In [10]:
# For each of the new features, encode each instance of each of the 7 words as either 0 (if not present) or 1 (if present) for the annotator sentiment
for index, row in MLMA_hate_speech_df.iterrows():
    for word in unique_annotator_sentiment:
        if word in row["annotator_sentiment"]:
            MLMA_hate_speech_df.at[index, f"{word}_annotator_sentiment"] = 1

In [11]:
MLMA_hate_speech_df.head()

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...,hateful_normal,indirect,shock,gender,individual,1,0,0,0,0,0,0
1,2,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other,0,1,1,1,1,1,0
2,3,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...,offensive,indirect,indifference,other,individual,0,0,0,0,1,0,0
3,4,@user @user انا اوافقك بخصوص السوريين و العراق...,normal,direct,indifference,origin,other,0,0,0,0,1,0,0
4,5,هذه السعودية التي شعبها شعب الخيم و بول البعير...,normal,indirect,indifference,origin,other,0,0,0,0,1,0,0


In [12]:
# Remove the "tweet" column
MLMA_hate_speech_df.pop("tweet")

,tweet
0,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...
1,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...
2,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...
3,@user @user انا اوافقك بخصوص السوريين و العراق...
4,هذه السعودية التي شعبها شعب الخيم و بول البعير...
...,...
18656,@user tres bien. le cinema francais c...est so...
18657,@user j’ai que des rebeus en demande d’amis aussi
18658,@user sale juif de merde
18659,@user pourquoi tant de migrants africains en e...


In [13]:
MLMA_hate_speech_df.head()

,HITId,sentiment,directness,annotator_sentiment,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,hateful_normal,indirect,shock,gender,individual,1,0,0,0,0,0,0
1,2,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other,0,1,1,1,1,1,0
2,3,offensive,indirect,indifference,other,individual,0,0,0,0,1,0,0
3,4,normal,direct,indifference,origin,other,0,0,0,0,1,0,0
4,5,normal,indirect,indifference,origin,other,0,0,0,0,1,0,0


In [14]:
# Drop the sentiment due to redundancy
MLMA_hate_speech_df.pop("annotator_sentiment")

,annotator_sentiment
0,shock
1,anger_confusion_sadness_indifference_disgust
2,indifference
3,indifference
4,indifference
...,...
18656,sadness
18657,indifference
18658,fear_shock_indifference_sadness_disgust
18659,indifference


In [15]:
MLMA_hate_speech_df.head()

,HITId,sentiment,directness,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,hateful_normal,indirect,gender,individual,1,0,0,0,0,0,0
1,2,offensive,indirect,other,other,0,1,1,1,1,1,0
2,3,offensive,indirect,other,individual,0,0,0,0,1,0,0
3,4,normal,direct,origin,other,0,0,0,0,1,0,0
4,5,normal,indirect,origin,other,0,0,0,0,1,0,0


In [16]:
# For sentiment, directness, target, and group, find the value counts:
for column in MLMA_hate_speech_df.columns:
    print(column, MLMA_hate_speech_df[column].value_counts())

HITId HITId
3353    4
3352    4
3351    4
3350    4
3349    4
       ..
4542    2
4543    2
4531    2
5647    1
0       1
Name: count, Length: 5648, dtype: int64
sentiment sentiment
offensive                               8194
normal                                  3058
hateful                                 1297
abusive                                  615
offensive_hateful                        554
                                        ... 
fearful_offensive_hateful                  2
abusive_hateful_disrespectful              2
fearful_offensive_disrespectful            2
abusive_hateful_disrespectful_normal       1
abusive_hateful_normal                     1
Name: count, Length: 69, dtype: int64
directness directness
indirect    13479
direct       5182
Name: count, dtype: int64
target target
origin                8039
other                 4948
disability            2356
gender                1851
sexual_orientation    1040
religion               427
Name: count, dtype: int64

In [17]:
# For sentiment, parse on _ and collect unique sentiment words
unique_sentiment = []
sentiment_word_count = 0

for word in MLMA_hate_speech_df["sentiment"].unique():
    words = word.split("_")
    for word in words:
        if word not in unique_sentiment:
            unique_sentiment.append(word)
            sentiment_word_count += 1

unique_sentiment, sentiment_word_count

(['hateful', 'normal', 'offensive', 'disrespectful', 'abusive', 'fearful'], 6)

In [18]:
# Create new columns using the unique_sentiment words
for word in unique_sentiment:
    MLMA_hate_speech_df[f"{word}_sentiment"] = 0

In [19]:
# For each entry, encode the presence of each unique sentiment word with 0 (for not present) and 1 (for present)
for index, row in MLMA_hate_speech_df.iterrows():
    for word in unique_sentiment:
        if word in row["sentiment"]:
            MLMA_hate_speech_df.at[index, f"{word}_sentiment"] = 1

In [20]:
# 0-1 encode directness
MLMA_hate_speech_df["directness"] = MLMA_hate_speech_df["directness"].map({"direct": 1, "indirect": 0})

In [21]:
MLMA_hate_speech_df.head()

,HITId,sentiment,directness,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,normal_sentiment,offensive_sentiment,disrespectful_sentiment,abusive_sentiment,fearful_sentiment
0,1,hateful_normal,0,gender,individual,1,0,0,0,0,0,0,1,1,0,0,0,0
1,2,offensive,0,other,other,0,1,1,1,1,1,0,0,0,1,0,0,0
2,3,offensive,0,other,individual,0,0,0,0,1,0,0,0,0,1,0,0,0
3,4,normal,1,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0
4,5,normal,0,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0


In [22]:
# Drop sentiment
MLMA_hate_speech_df.pop("sentiment")

,sentiment
0,hateful_normal
1,offensive
2,offensive
3,normal
4,normal
...,...
18656,offensive
18657,disrespectful_normal
18658,offensive
18659,normal


In [23]:
MLMA_hate_speech_df.head()

,HITId,directness,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,normal_sentiment,offensive_sentiment,disrespectful_sentiment,abusive_sentiment,fearful_sentiment
0,1,0,gender,individual,1,0,0,0,0,0,0,1,1,0,0,0,0
1,2,0,other,other,0,1,1,1,1,1,0,0,0,1,0,0,0
2,3,0,other,individual,0,0,0,0,1,0,0,0,0,1,0,0,0
3,4,1,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0
4,5,0,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0


In [24]:
unique_targets = MLMA_hate_speech_df["target"].unique()

In [25]:
unique_groups = MLMA_hate_speech_df["group"].unique()

In [26]:
# Combine the unique target and unique group values into a new "target_group" feature
MLMA_hate_speech_df["target_group"] = MLMA_hate_speech_df["target"].astype(str) + "_" + MLMA_hate_speech_df["group"].astype(str)

In [27]:
unique_target_groups = MLMA_hate_speech_df["target_group"].unique()
unique_target_groups

array(['gender_individual', 'other_other', 'other_individual',
       'origin_other', 'gender_women', 'other_women', 'other_arabs',
       'gender_other', 'origin_individual', 'origin_arabs',
       'other_indian/hindu', 'origin_refugees', 'other_refugees',
       'other_african_descent', 'origin_muslims', 'other_special_needs',
       'origin_women', 'religion_indian/hindu', 'religion_other',
       'origin_asians', 'other_immigrants', 'religion_individual',
       'religion_muslims', 'other_muslims', 'origin_african_descent',
       'other_jews', 'religion_christian', 'other_gay', 'other_christian',
       'religion_jews', 'origin_indian/hindu', 'gender_gay',
       'disability_special_needs', 'origin_jews', 'religion_arabs',
       'origin_gay', 'origin_left_wing_people',
       'sexual_orientation_other', 'origin_special_needs',
       'sexual_orientation_individual', 'sexual_orientation_women',
       'origin_immigrants', 'disability_women', 'gender_refugees',
       'sexual_orien

In [28]:
MLMA_hate_speech_df["target_group"].value_counts()

,count
target_group,
origin_other,2767
other_other,2285
disability_special_needs,1972
gender_women,1402
other_individual,1241
...,...
disability_immigrants,2
disability_indian/hindu,2
religion_arabs,1


In [29]:
# Sort the target group alphabetically
MLMA_hate_speech_df = MLMA_hate_speech_df.sort_values(by="target_group")
MLMA_hate_speech_df

,HITId,directness,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,normal_sentiment,offensive_sentiment,disrespectful_sentiment,abusive_sentiment,fearful_sentiment,target_group
14091,5091,0,disability,african_descent,1,0,0,1,0,0,0,0,0,1,0,0,0,disability_african_descent
6264,2912,0,disability,african_descent,0,1,0,0,1,0,0,1,0,1,1,1,1,disability_african_descent
11244,2244,0,disability,african_descent,0,1,0,0,1,0,0,1,0,1,1,1,1,disability_african_descent
8243,4891,0,disability,african_descent,1,0,0,1,0,0,0,0,0,1,0,0,0,disability_african_descent
12610,3610,0,disability,gay,0,1,0,0,0,1,0,0,0,0,1,1,0,disability_gay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8540,5188,0,sexual_orientation,women,0,0,0,0,0,1,1,1,0,1,1,1,1,sexual_orientation_women
5617,2265,0,sexual_orientation,women,0,1,0,0,0,1,0,1,0,1,0,0,0,sexual_orientation_women
5650,2298,1,sexual_orientation,women,0,1,0,0,0,1,0,0,0,1,0,0,0,sexual_orientation_women
12457,3457,0,sexual_orientation,women,0,1,0,0,0,1,0,1,0,1,0,0,0,sexual_orientation_women


In [30]:
# Get the unique values of target_group
unique_target_groups = MLMA_hate_speech_df["target_group"].unique()
unique_target_groups

array(['disability_african_descent', 'disability_gay',
       'disability_hispanics', 'disability_immigrants',
       'disability_indian/hindu', 'disability_individual',
       'disability_jews', 'disability_left_wing_people',
       'disability_other', 'disability_refugees',
       'disability_special_needs', 'disability_women',
       'gender_african_descent', 'gender_gay', 'gender_immigrants',
       'gender_individual', 'gender_left_wing_people', 'gender_muslims',
       'gender_other', 'gender_refugees', 'gender_special_needs',
       'gender_women', 'origin_african_descent', 'origin_arabs',
       'origin_asians', 'origin_christian', 'origin_gay',
       'origin_hispanics', 'origin_immigrants', 'origin_indian/hindu',
       'origin_individual', 'origin_jews', 'origin_left_wing_people',
       'origin_muslims', 'origin_other', 'origin_refugees',
       'origin_special_needs', 'origin_women', 'other_african_descent',
       'other_arabs', 'other_asians', 'other_christian', 'other_g

In [31]:
# Encode the target_group values with their unique index positions:
for i, target_group in enumerate(unique_target_groups):
  # Select all instances of the target group
  MLMA_hate_speech_df[MLMA_hate_speech_df["target_group"] == target_group] = i

In [32]:
MLMA_hate_speech_df["target_group"].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78], dtype=object)

In [33]:
MLMA_hate_speech_df = MLMA_hate_speech_df.drop(columns=["target", "group"])

In [34]:
MLMA_hate_speech_df.head()

,HITId,directness,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,normal_sentiment,offensive_sentiment,disrespectful_sentiment,abusive_sentiment,fearful_sentiment,target_group
14091,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6264,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11244,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12610,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [35]:
MLMA_hate_speech_df = MLMA_hate_speech_df.reset_index()

In [36]:
# Output MLMA_hate_speech_df as MLMA_hate_speech_full.csv
MLMA_hate_speech_df.to_csv("/content/drive/My Drive/Online MSDS/MOD C2/Political Polarization/data/MLMA_hate_speech_full.csv", index=False)

In [37]:
# Remove the index column
MLMA_hate_speech_df = MLMA_hate_speech_df.drop("index", axis=1)
MLMA_hate_speech_df

,HITId,directness,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,normal_sentiment,offensive_sentiment,disrespectful_sentiment,abusive_sentiment,fearful_sentiment,target_group
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18656,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78
18657,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78
18658,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78
18659,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78


In [38]:
# Output MLMA_hate_speech_df as MLMA_hate_speech_imbalanced.csv
MLMA_hate_speech_df.to_csv("/content/drive/My Drive/Online MSDS/MOD C2/Political Polarization/data/MLMA_hate_speech_imbalanced.csv", index=False)

In [39]:
len(unique_target_groups)

79

In [40]:
unique_target_group_indices = MLMA_hate_speech_df["target_group"].unique()

In [41]:
# Create a dataframe using the unique target groups
unique_target_groups_df = pd.DataFrame({
    "target_group": unique_target_groups,
    "index": unique_target_group_indices
})

unique_target_groups_df

,target_group,index
0,disability_african_descent,0
1,disability_gay,1
2,disability_hispanics,2
3,disability_immigrants,3
4,disability_indian/hindu,4
...,...,...
74,sexual_orientation_muslims,74
75,sexual_orientation_other,75
76,sexual_orientation_refugees,76
77,sexual_orientation_special_needs,77


In [42]:
MLMA_hate_speech_df["target_group"].value_counts()

,count
target_group,
34,2767
49,2285
10,1972
21,1402
45,1241
...,...
56,2
69,2
57,1


In [43]:
unique_target_groups_df.to_csv("/content/drive/My Drive/Online MSDS/MOD C2/Political Polarization/data/MLMA_target_groups.csv", index=False)

In [44]:
# Upsample the minority target groups
majority_target_group = MLMA_hate_speech_df[MLMA_hate_speech_df["target_group"] == 34]
majority_target_group_length = len(majority_target_group)
majority_target_group_length

2767

In [49]:
# Upsampled data
MLMA_hate_speech_df_upsampled_list = []

for current_group_index in unique_target_group_indices:
  current_group_data = MLMA_hate_speech_df[MLMA_hate_speech_df["target_group"] == current_group_index]

  if current_group_index == 34:
    MLMA_hate_speech_df_upsampled_list.append(current_group_data)
    continue

  # Upsample the minority class
  current_group_upsampled = resample(current_group_data, random_state=42, n_samples=majority_target_group_length, replace=True)
  MLMA_hate_speech_df_upsampled_list.append(current_group_upsampled)

In [50]:
# Combine all data in the MLMA_hate_speech_df_upsampled_list into a single dataframe
MLMA_hate_speech_df_upsampled = pd.concat([sample for sample in MLMA_hate_speech_df_upsampled_list])

In [53]:
len(MLMA_hate_speech_df_upsampled), len(MLMA_hate_speech_df)

(218593, 18661)

In [55]:
MLMA_hate_speech_df_upsampled.to_csv("/content/drive/My Drive/Online MSDS/MOD C2/Political Polarization/data/MLMA_hate_speech_final.csv", index=False)